In [2]:
from itertools import chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_weather(lon, lat):
    print("call an api....")

schema = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "The longitude coordinate",
            },
            "lat": {
                "type": "string",
                "description": "The latitude coordinate",
            },
        },
    },
    "required": [
        "lon",
        "lat",
    ],
}

llm = ChatOpenAI(
    temperature=0.1,
).bind()
prompt = PromptTemplate.from_template("Who is the weather in {city}")

chain = prompt | llm

chain.invoke({
    "city": "rome",
})

SyntaxError: ':' expected after dictionary key (1597230551.py, line 13)